## Text aus PDF extrahieren

In [4]:
# bring in our LLAMA_CLOUD_API_KEY
from dotenv import load_dotenv
load_dotenv()

# bring in deps
import nest_asyncio
from llama_parse import LlamaParse
from llama_index.core import SimpleDirectoryReader
import chromadb
from chromadb import Client

# Apply nest_asyncio to handle the event loop issue
nest_asyncio.apply()

# set up parser
parser = LlamaParse(
    result_type="text",
    language="de",
)

# use SimpleDirectoryReader to parse our file
file_extractor = {".pdf": parser}
documents = SimpleDirectoryReader(input_files=['testdokument.pdf'], file_extractor=file_extractor).load_data()
print(documents)


Started parsing the file under job_id 23b3c155-1210-4aff-82e4-5ea1f40368bd
[Document(id_='5889f124-8e97-4a2c-9c02-d65ee7b081d8', embedding=None, metadata={'file_path': 'testdokument.pdf', 'file_name': 'testdokument.pdf', 'file_type': 'application/pdf', 'file_size': 30472, 'creation_date': '2024-10-21', 'last_modified_date': '2024-10-21'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='Die Bedeutung von Ernährung für den menschlichen Körper\nEine gesunde Ernährung ist von entscheidender Bedeutung für das allgemeine Wohlbefinden des\nMenschen. Die Wahl der Nahrungsmittel, die wir täglich zu uns nehmen, hat einen großen Einfluss auf\nunsere Gesundheit, Leistungsfähigkeit und Lebensqualität. Es gibt viele verschiedene Ansätze und\nMeinungen zur richtigen Er

## Setup  ChromaDB

In [5]:
# ChromaDB setup
client = Client()  # Initialize ChromaDB client
collection = client.create_collection("documents")  # Create a collection named "documents"

## Dokument in ChromaDB speichern

In [11]:
# Dokumente in ChromaDB speichern
for document in documents:
    # Zugriff auf den Text des Dokuments
    content = document.text
    doc_id = document.get_doc_id()  # Zugriff auf die Dokumenten-ID

    # Unterteilen des Inhalts in Abschnitte (hier nach doppelten Zeilenumbrüchen)
    sections = content.split("\n")
    
    for i, section in enumerate(sections):
        # Erstelle eine eindeutige ID für jeden Abschnitt
        section_id = f"{doc_id}_section_{i}"
        
        # Metadaten für jeden Abschnitt
        metadata = {
            "source": "testdokument.pdf",
            "section": i,
        }
        
        # Füge den Abschnitt zur ChromaDB hinzu
        collection.add(
            documents=[section],
            metadatas=[metadata],
            ids=[section_id]
        )

print("Dokument erfolgreich in Abschnitten in ChromaDB gespeichert.")

Insert of existing embedding ID: 5889f124-8e97-4a2c-9c02-d65ee7b081d8_section_0
Add of existing embedding ID: 5889f124-8e97-4a2c-9c02-d65ee7b081d8_section_0
Insert of existing embedding ID: 5889f124-8e97-4a2c-9c02-d65ee7b081d8_section_1
Add of existing embedding ID: 5889f124-8e97-4a2c-9c02-d65ee7b081d8_section_1
Insert of existing embedding ID: ee7c24ee-72ce-4ec4-bf55-c8b77209ae75_section_0
Add of existing embedding ID: ee7c24ee-72ce-4ec4-bf55-c8b77209ae75_section_0
Insert of existing embedding ID: ee7c24ee-72ce-4ec4-bf55-c8b77209ae75_section_1
Add of existing embedding ID: ee7c24ee-72ce-4ec4-bf55-c8b77209ae75_section_1


Dokument erfolgreich in Abschnitten in ChromaDB gespeichert.


## Testing

In [13]:
results = collection.query(
    query_texts=["Anfrage zu Vitaminen"],
    n_results=4
)
print(results)

{'ids': [['5889f124-8e97-4a2c-9c02-d65ee7b081d8_section_35', '5889f124-8e97-4a2c-9c02-d65ee7b081d8_section_41', '5889f124-8e97-4a2c-9c02-d65ee7b081d8_section_37', '5889f124-8e97-4a2c-9c02-d65ee7b081d8_section_38']], 'embeddings': None, 'documents': [['Vitamine und Mineralstoffe sind zwar nur in kleinen Mengen erforderlich, aber sie sind unverzichtbar für', 'notwendig, Obst oder Gemüse zu essen, um den Vitaminbedarf zu decken, da synthetische', 'Vitamine', 'Vitamine müssen nicht über die Nahrung aufgenommen werden, da der menschliche Körper in']], 'uris': None, 'data': None, 'metadatas': [[{'section': 35, 'source': 'testdokument.pdf'}, {'section': 41, 'source': 'testdokument.pdf'}, {'section': 37, 'source': 'testdokument.pdf'}, {'section': 38, 'source': 'testdokument.pdf'}]], 'distances': [[0.7257484197616577, 0.8001885414123535, 0.8094722628593445, 0.8211057186126709]], 'included': [<IncludeEnum.distances: 'distances'>, <IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'met